# Lattice models

## Introduction
In quantum physics (including condensed matter physics and high energy physics) we often study models on lattices. 
For example, when we think about behavior of electrons in a solid, we can study a model defined on a lattice by considering the positions of atoms as lattice points.
This notebook demonstrates how we can utilize `Lattice` classes to generate various lattice systems such as `LineLattice`, `SquareLattice`, `HyperCubicLattice`, `TriangularLattice`, and a general lattice. 
It also includes an example of a lattice model, the Fermi-Hubbard model. 
We see how we can define the Hamiltonian of the Fermi-Hubbard model for a given lattice using `FermiHubbardModel` class. 

In [ ]:
from math import pi

import numpy as np
import retworkx as rx
from qiskit_nature.problems.second_quantization.lattice import (
    BoundaryCondition,
    FermiHubbardModel,
    HyperCubicLattice,
    Lattice,
    LatticeDrawStyle,
    LineLattice,
    SquareLattice,
    TriangularLattice,
)

## LineLattice
`LineLattice` provides a one-dimensional lattice.
We can construct a one-dimensional lattice as follows.

In [ ]:
num_nodes = 11
boundary_condition = BoundaryCondition.OPEN
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

Here, it is visualized.

In [ ]:
line_lattice.draw()

We can also construct a one-dimensional lattice with the periodic boundary conditions by specifying `BoundaryCondition.PERIODIC` as the argument of `boundary_condition`.

In [ ]:
num_nodes = 11
boundary_condition = BoundaryCondition.PERIODIC
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

line_lattice.draw()

When we want to draw the lattice ignoring the boundary conditions, we use the method `draw_without_boundary`.

In [ ]:
line_lattice.draw_without_boundary()

We can define real or complex weights for the edges of the lattice.
This is done by giving a value to the argument `edge_parameter`.
We can also give a value for the self-loops of the lattice by passing the value for `onsite_parameter`.

In [ ]:
num_nodes = 11
boundary_condition = BoundaryCondition.PERIODIC
edge_parameter = 1.0 + 1.0j
onsite_parameter = 1.0
line_lattice = LineLattice(
    num_nodes=num_nodes,
    edge_parameter=edge_parameter,
    onsite_parameter=onsite_parameter,
    boundary_condition=boundary_condition,
)
set(line_lattice.graph.weighted_edge_list())

The connectivity of the lattice can be seen as the adjacency matrix, which is done by `to_adjacency_matrix`.

In [ ]:
line_lattice.to_adjacency_matrix()

By setting `weighted=True`, we obtain a Hermitian matrix whose matrix elements are the weights.

In [ ]:
line_lattice.to_adjacency_matrix(weighted=True)

## SquareLattice
`SquareLattice` provides a two-dimensional lattice.
Here, we make a two-dimensional lattice with the open boundary conditions.

In [ ]:
rows = 5
cols = 4
boundary_condition = BoundaryCondition.OPEN
square_lattice = SquareLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

square_lattice.draw()

We can specify the boundary conditions for each direction by setting `boundary_condition` as a tuple.

In [ ]:
rows = 5
cols = 4
boundary_condition = (
    BoundaryCondition.OPEN,
    BoundaryCondition.PERIODIC,
)  # open in the x-direction, periodic in the y-direction
square_lattice = SquareLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

square_lattice.draw()

Again, we can give weights on the edges and the self-loops of the lattice.
Here, it is possible to give weights for each direction as a tuple.

In [ ]:
rows = 5
cols = 4
edge_parameter = (1.0, 1.0 + 1.0j)
boundary_condition = (
    BoundaryCondition.OPEN,
    BoundaryCondition.PERIODIC,
)  # open in the x-direction, periodic in the y-direction
onsite_parameter = 1.0
square_lattice = SquareLattice(
    rows=rows,
    cols=cols,
    edge_parameter=edge_parameter,
    onsite_parameter=onsite_parameter,
    boundary_condition=boundary_condition,
)

set(square_lattice.graph.weighted_edge_list())

## HyperCubicLattice
`HyperCubicLattice` is a generalization of `LineLattice` and `SquareLattice`. 
It provides an arbitrary d-dimensional lattice.
Here, we make a three-dimensional lattice of size 3 by 4 by 5 as an example.
The size is given as a tuple, and the boundary conditions can be specified for each direction too.
In the example, the boundary conditions are open.

In [ ]:
size = (3, 4, 5)
boundary_condition = (
    BoundaryCondition.OPEN,
    BoundaryCondition.OPEN,
    BoundaryCondition.OPEN,
)
cubic_lattice = HyperCubicLattice(size=size, boundary_condition=boundary_condition)

We draw the cubic lattice specifying the positions of the lattice points.

In [ ]:
# function for setting the positions
def indextocoord_3d(index: int, size: tuple, angle) -> list:
    z = index // (size[0] * size[1])
    a = index % (size[0] * size[1])
    y = a // size[0]
    x = a % size[0]
    vec_x = np.array([1, 0])
    vec_y = np.array([np.cos(angle), np.sin(angle)])
    vec_z = np.array([0, 1])
    return_coord = x * vec_x + y * vec_y + z * vec_z
    return return_coord.tolist()


pos = dict([(index, indextocoord_3d(index, size, angle=pi / 4)) for index in range(np.prod(size))])

cubic_lattice.draw(style=LatticeDrawStyle(pos=pos))

## TriangularLattice
`TriangularLattice` provides a triangular lattice, which can be seen as a two-dimensional lattice with diagonal edges.
The argument `boundary_condition` can take either "open" or "periodic".

In [ ]:
rows = 4
cols = 3
boundary_condition = BoundaryCondition.OPEN

triangular_lattice = TriangularLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

triangular_lattice.draw()

In [ ]:
rows = 4
cols = 3
boundary_condition = BoundaryCondition.PERIODIC

triangular_lattice = TriangularLattice(rows=rows, cols=cols, boundary_condition=boundary_condition)

triangular_lattice.draw()

## General Lattice
Above, we have seen translational invariant lattices.
Here, we consider a general lattice.
We can construct a general lattice consisting of nodes and edges using an instance of PyGraph.

In [ ]:
graph = rx.PyGraph(multigraph=False)  # multiigraph shoud be False
graph.add_nodes_from(range(6))
weighted_edge_list = [
    (0, 1, 1.0 + 1.0j),
    (0, 2, -1.0),
    (2, 3, 2.0),
    (4, 2, -1.0 + 2.0j),
    (4, 4, 3.0),
    (2, 5, -1.0),
]
graph.add_edges_from(weighted_edge_list)

# make a lattice
general_lattice = Lattice(graph)
set(general_lattice.graph.weighted_edge_list())

Here is its visualization.

In [ ]:
general_lattice.draw()

When we want to visualize the self-loops in the lattice, we set `self_loop` to True.

In [ ]:
general_lattice.draw(self_loop=True)

The labels of the lattice sites are drawn when `with_labels` is True.

In [ ]:
general_lattice.draw(self_loop=True, style=LatticeDrawStyle(with_labels=True))

## The Fermi-Hubbard model
The Fermi-Hubbard model is the simplest model describing electrons moving on a lattice and interaction with each other at the same site.
The Hamiltonian is given as follows:

$$
H = \sum_{i, j}\sum_{\sigma = \uparrow, \downarrow} t_{i, j} c_{i, \sigma}^\dagger c_{j, \sigma} + U \sum_{i} n_{i, \uparrow} n_{i, \downarrow},
$$

where $c_{i, \sigma}^\dagger$ and $c_{i, \sigma}$ are creation and annihilation operators of fermion at the site $i$ with spin $\sigma$.
The operator $n_{i, \sigma}$ is the number operator, which is defined by $n_{i, \sigma} = c_{i, \sigma}^\dagger c_{i, \sigma}$. 
The matrix $t_{i, j}$ is a Hermitian matrix called interaction matrix.
The parameter $U$ represents the strength of the interaction.

We can generate the corresponding Hamiltonian of a given lattice using `FermiHubbardModel` class.
Here, we construct the Hamiltonian with uniform interaction and interaction parameters on a two-dimensional lattice.

In [ ]:
square_lattice = SquareLattice(rows=5, cols=4, boundary_condition=BoundaryCondition.PERIODIC)

t = -1.0  # the interaction parameter
v = 0.0  # the onsite potential
u = 5.0  # the interaction parameter U

fhm = FermiHubbardModel.uniform_parameters(
    lattice=square_lattice,
    uniform_interaction=t,
    uniform_onsite_potential=v,
    onsite_interaction=u,
)

To obtain the Hamiltonian in terms of the fermionic operators, we use `second_q_ops` method.
The Hamiltonian is returned as an instance of `FermionicOp`.

- Note
    - The number of fermionic operators required is twice the number of lattice sites because of the spin degrees of freedom.
    - In the implementation, even indexes correspond to up-spin and odd indexes to down-spin.

In [ ]:
ham = fhm.second_q_ops(display_format="sparse").reduce()
print(ham)

`Lattice` has weights on its edges, so we can define a general interaction matrix using a Lattice instance.
Here, we consider the Fermi-Hubbard model on a general lattice on which non-uniform interaction parameters are given.
In this case, the weights of the lattice are regarded as the interaction matrix. After generating the Hamiltonian (`second_q_ops`) we can use a qubit converter to generate the qubit operators and/or use any of the available algorithms to solver the corresponding lattice problem.

In [ ]:
graph = rx.PyGraph(multigraph=False)  # multiigraph shoud be False
graph.add_nodes_from(range(6))
weighted_edge_list = [
    (0, 1, 1.0 + 1.0j),
    (0, 2, -1.0),
    (2, 3, 2.0),
    (4, 2, -1.0 + 2.0j),
    (4, 4, 3.0),
    (2, 5, -1.0),
]
graph.add_edges_from(weighted_edge_list)

general_lattice = Lattice(graph)  # the lattice whose weights are seen as the interaction matrix.
u = 5.0  # the interaction parameter U

fhm = FermiHubbardModel(lattice=general_lattice, onsite_interaction=u)

ham = fhm.second_q_ops(display_format="sparse").reduce()
ham.set_truncation(0)
print(ham)

## LatticeModelProblem
We also have the Lattice Model Problem to generate the Second quantized operator from a Lattice Model. We can use the Ground State Eigensolver with it to calculate the ground state energy.

In [ ]:
from qiskit_nature.problems.second_quantization import LatticeModelProblem
from qiskit_nature.converters.second_quantization import QubitConverter

num_nodes = 4
boundary_condition = BoundaryCondition.OPEN
line_lattice = LineLattice(num_nodes=num_nodes, boundary_condition=boundary_condition)

fhm = FermiHubbardModel.uniform_parameters(
    lattice=line_lattice,
    uniform_interaction=t,
    uniform_onsite_potential=v,
    onsite_interaction=u,
)

lmp = LatticeModelProblem(lattice_model=fhm)

In [ ]:
from qiskit.algorithms import NumPyMinimumEigensolver
from qiskit_nature.algorithms import GroundStateEigensolver
from qiskit_nature.mappers.second_quantization import JordanWignerMapper

numpy_solver = NumPyMinimumEigensolver()

qubit_converter = QubitConverter(JordanWignerMapper())

calc = GroundStateEigensolver(qubit_converter, numpy_solver)
res = calc.solve(lmp)

print(res)

In [ ]:
import qiskit.tools.jupyter

%qiskit_version_table
%qiskit_copyright